In [2]:
import os
import pandas
import numpy
import pickle
import pefile
import sklearn.ensemble as ek
from sklearn.tree import ExtraTreeClassifier
from sklearn import cross_validation, tree, linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression

In [3]:
dataset = pandas.read_csv('fbvhbvfh.csv',sep=',', low_memory=False)

In [4]:
dataset.head()

,Station_Number,Station_name,Depth_to_water_level_feet_below_LSD,Elevation_feet_above_NGVD,Ground_water_level_above_NAVD_feet,Date/Time
0,Alabama,NaN,NaN,NaN,NaN,NaN
1,302416087505501,ww 13-usgs 302416087505501,3.25,0.0,0,08/29 06:00 CDT
2,311319086153601,COVINGTON COUNTY WELL 1,NaN,NaN,NaN,NaN
3,,[(2)],11.12,0.0,0,08/29 06:00 CDT
4,322047086214301,K 107 MTG-3,24.12,0.0,0,08/29 05:00 CDT


In [5]:
dataset.fillna(0, inplace=True)

In [6]:
dataset.head()

,Station_Number,Station_name,Depth_to_water_level_feet_below_LSD,Elevation_feet_above_NGVD,Ground_water_level_above_NAVD_feet,Date/Time
0,Alabama,0,0.00,0.0,0,0
1,302416087505501,ww 13-usgs 302416087505501,3.25,0.0,0,08/29 06:00 CDT
2,311319086153601,COVINGTON COUNTY WELL 1,0.00,0.0,0,0
3,,[(2)],11.12,0.0,0,08/29 06:00 CDT
4,322047086214301,K 107 MTG-3,24.12,0.0,0,08/29 05:00 CDT


In [7]:
dataset.describe()

,Depth_to_water_level_feet_below_LSD,Elevation_feet_above_NGVD
count,1863.000000,1863.0
mean,72.043473,0.0
std,112.182843,0.0
min,-74.410000,0.0
25%,9.050000,0.0
50%,26.330000,0.0
75%,85.320000,0.0
max,979.810000,0.0


In [8]:
import numpy as np
from decimal import Decimal
print(type(dataset['Depth_to_water_level_feet_below_LSD']))
dataset['result'] = np.where(dataset['Depth_to_water_level_feet_below_LSD'].astype(float)>=20, 1, 0)
p, n = dataset.groupby(dataset['result']).size()
print(p, n)

<class 'pandas.core.series.Series'>
795 1068


In [9]:
dataset.head()

,Station_Number,Station_name,Depth_to_water_level_feet_below_LSD,Elevation_feet_above_NGVD,Ground_water_level_above_NAVD_feet,Date/Time,result
0,Alabama,0,0.00,0.0,0,0,0
1,302416087505501,ww 13-usgs 302416087505501,3.25,0.0,0,08/29 06:00 CDT,0
2,311319086153601,COVINGTON COUNTY WELL 1,0.00,0.0,0,0,0
3,,[(2)],11.12,0.0,0,08/29 06:00 CDT,0
4,322047086214301,K 107 MTG-3,24.12,0.0,0,08/29 05:00 CDT,1


In [10]:
X = dataset.drop(['Station_Number','Station_name','Elevation_feet_above_NGVD', 'Ground_water_level_above_NAVD_feet', 'Date/Time', 'result'],axis=1).values
# X = dataset.values
y = dataset['result'].values

In [11]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y ,test_size=0.2)

In [12]:
#using RFC
clf = ek.RandomForestClassifier(n_estimators=50)
labels = ('Dense', 'Rare')
clf.fit(X_train, y_train)
score = clf.score(X_test,y_test)
score
# using knn
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train) 
knn.predict(X_test)
knn.score(X_test, y_test)

1.0

In [13]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
regr.coef_
np.mean((regr.predict(X_test)-y_test)**2)

0.18791009949126

In [14]:
regr.score(X_test, y_test) 

0.22907100046828519

In [15]:
y_pred = regr.predict(X_test)
regr.intercept_

0.4123123237388695

In [16]:
import pandas as pd
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

,Actual,Predicted
0,1,1.059774
1,0,0.430487
2,1,0.501533
3,1,0.792654
4,0,0.424922
5,1,1.433202
6,1,0.462901
7,1,0.466162
8,0,0.412312
9,0,0.430748


In [17]:
from sklearn import metrics 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))  

Mean Absolute Error: 0.41651152130793423
Mean Squared Error: 0.18791009949126
Root Mean Squared Error: 0.43348598534584715


In [18]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2")
clf.fit(X, y)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [19]:
res = clf.predict([[56.261251254545]])[0]
labels = ('Rare', 'Dense')
labels[res]

'Dense'

In [21]:
res = clf.predict([[16.261251254545]])[0]
labels = ('Rare', 'Dense')
labels[res]

'Rare'